In [20]:
import torch
import pickle
import numpy as np
import pandas as pd
from argparse import Namespace
from torch.utils.data import DataLoader
import torch.nn.functional as F
from heterogt.utils.tokenizer import EHRTokenizer
from heterogt.utils.dataset import FineTuneEHRDataset, batcher, expand_level3
from heterogt.utils.train import train_with_early_stopping, PHENO_ORDER
from heterogt.utils.seed import set_random_seed
from heterogt.model.model import HeteroGTFineTune

In [2]:
set_random_seed(123)

[INFO] Random seed set to 123


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
config = Namespace(
    dataset = "MIMIC-III",
    tasks = ["death", "readmission", "stay", "next_diag_6m", "next_diag_12m"],
    task_index = 4,  # index of the task to train
    token_type = ["diag", "med", "lab", "pro"],
    special_tokens = ["[PAD]", "[CLS]"],
    # only for tf layer
    attn_mask_dicts = [{1:[6,7], 2:[6,7], 3:[6,7], 4:[6,7], 5:[6,7], 6:[1,2,3,4,5,6,7], 7:[1,2,3,4,5,6,7]}, 
                       {1:[6,7], 2:[6,7], 3:[6,7], 4:[6,7], 5:[6,7], 6:[1,2,3,4,5,6,7], 7:[1,2,3,4,5,6,7]}], 
    d_model = 64,
    num_heads = 4,
    batch_size = 32,
    lr = 1e-3,
    epochs = 500,
    early_stop_patience = 5,
    group_code_thre = 5,  # if there are group_code_thre diag codes belongs to the same group ICD code, then the group code is generated
    use_pretrained_model = True,
    pretrain_mask_rate = 0.7,
    pretrain_cls_ontology_weight = 5e-2,
    pretrain_visit_ontology_weight = 5e-2,
    pretrain_adm_type_weight = 0,
    dec_loss_lambda = 1e-2,
)

In [5]:
full_data_path = f"./data_process/{config.dataset}-processed/mimic.pkl"  # for tokenizer
curr_task = config.tasks[config.task_index]
print("Current task:", curr_task)
if curr_task == "next_diag_6m":
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_nextdiag_6m.pkl"
elif curr_task == "next_diag_12m":
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_nextdiag_12m.pkl"
else:
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_downstream.pkl"

Current task: next_diag_12m


In [6]:
ehr_full_data = pickle.load(open(full_data_path, 'rb'))
group_code_sentences = [expand_level3()[1]]
diag_sentences = ehr_full_data["ICD9_CODE"].values.tolist()
med_sentences = ehr_full_data["NDC"].values.tolist()
lab_sentences = ehr_full_data["LAB_TEST"].values.tolist()
pro_sentences = ehr_full_data["PRO_CODE"].values.tolist()
age_sentences = [[str(c)] for c in set(ehr_full_data["AGE"].values.tolist())] # important of [[]]
adm_type_sentences = ehr_full_data["ADMISSION_TYPE"].values.tolist()
max_admissions = ehr_full_data.groupby("SUBJECT_ID")["HADM_ID"].nunique().max()
config.max_num_adms = max_admissions
print(f"Max admissions per patient: {config.max_num_adms}")

Max admissions per patient: 8


In [7]:
tokenizer = EHRTokenizer(age_sentences, group_code_sentences, diag_sentences, med_sentences, lab_sentences, 
                         pro_sentences, special_tokens=config.special_tokens, adm_types_sentences=adm_type_sentences)
config.label_vocab_size = len(PHENO_ORDER)  # a set of predifined diseases
config.global_vocab_size = len(tokenizer.vocab.id2word)
config.age_vocab_size = tokenizer.token_number("age")
config.group_code_vocab_size = tokenizer.token_number("group")
print(f"Age vocabulary size: {config.age_vocab_size}")
print(f"Group code vocabulary size: {config.group_code_vocab_size}")

Age vocabulary size: 18
Group code vocabulary size: 19


In [8]:
train_data, val_data, test_data = pickle.load(open(finetune_data_path, 'rb'))
# example label percentage
print("Percentage of DEATH in test dataset:",
      (test_data["DEATH"] == True).mean() * 100, "%")

print("Percentage of READMISSION in test dataset:",
      (test_data["READMISSION"] == 1).mean() * 100, "%")

print("Percentage of STAY>7 days in test dataset:",
      (test_data["STAY_DAYS"] > 7).mean() * 100, "%")

Percentage of DEATH in test dataset: 8.114199849737041 %
Percentage of READMISSION in test dataset: 64.08715251690458 %
Percentage of STAY>7 days in test dataset: 55.10894064613073 %


In [9]:
train_dataset = FineTuneEHRDataset(train_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                   max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)
val_dataset = FineTuneEHRDataset(val_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                 max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)
test_dataset = FineTuneEHRDataset(test_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                   max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)

In [10]:
long_adm_seq_crite = 3
val_long_seq_idx, test_long_seq_idx = [], []
for i in range(len(val_dataset)):
    hadm_id = list(val_dataset.records.keys())[i]
    num_adms = len(val_dataset.records[hadm_id])
    if num_adms >= long_adm_seq_crite:
        val_long_seq_idx.append(i)
for i in range(len(test_dataset)):
    hadm_id = list(test_dataset.records.keys())[i]
    num_adms = len(test_dataset.records[hadm_id])
    if num_adms >= long_adm_seq_crite:
        test_long_seq_idx.append(i)
print(len(val_long_seq_idx), len(test_long_seq_idx))

208 186


In [11]:
num_group_code = []
for i in range(len(train_dataset)):
    input_ids, token_types, adm_index, age_ids, diag_group_codes, labels = train_dataset[i]
    count = (token_types[0] == 6).sum().item()
    num_group_code.append(count)
print("Mean group token numer per patient", np.mean(num_group_code))

Mean group token numer per patient 0.8268720127456187


In [12]:
train_dataloader = DataLoader(
    train_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=True,
    batch_size=config.batch_size,
)

val_dataloader = DataLoader(
    val_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=False,
    batch_size=config.batch_size,
)

test_dataloader = DataLoader(
    test_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=False,
    batch_size=config.batch_size,
)

In [13]:
for batch in train_dataloader:
    pass  # just to check if the dataloader works
for batch in val_dataloader:
    pass  # just to check if the dataloader works
for batch in test_dataloader:
    pass  # just to check if the dataloader works
print("All pass!")

All pass!


In [14]:
if curr_task in ["death", "stay", "readmission"]:
    eval_metric = "f1"
    task_type = "binary"
    loss_fn = F.binary_cross_entropy_with_logits
else:
    eval_metric = "f1"
    task_type = "l2r"
    loss_fn = lambda x, y: F.binary_cross_entropy_with_logits(x, y)

In [15]:
input_ids, token_types, adm_index, age_ids, diag_code_group_dicts, labels = next(iter(train_dataloader))
print("Input IDs shape:", input_ids.shape)
print("Token Types shape:", token_types.shape)
print("Admission Index shape:", adm_index.shape)
print("Age IDs shape:", age_ids.shape)
print("Diag Code Group Dict number:", len(diag_code_group_dicts))
print("Labels shape:", labels.shape)

Input IDs shape: torch.Size([32, 193])
Token Types shape: torch.Size([32, 193])
Admission Index shape: torch.Size([32, 193])
Age IDs shape: torch.Size([32, 4])
Diag Code Group Dict number: 32
Labels shape: torch.Size([32, 18])


# Model Walkthrough

In [16]:
# load pretrained model
if config.use_pretrained_model:
    pretrain_exp_name = (
    f"{config.dataset}-{config.pretrain_mask_rate}-{config.d_model}-{config.pretrain_cls_ontology_weight}-{config.pretrain_visit_ontology_weight}-{config.pretrain_adm_type_weight}"
)
    print(pretrain_exp_name)
    save_path = "./pretrained_models/" + pretrain_exp_name
    state_dict = torch.load(f"{save_path}/pretrained_model.pt", map_location="cpu")

MIMIC-III-0.7-64-0.05-0.05-0


In [ ]:
final_metrics, final_long_seq_metrics = [], []
for i in range(15):
    print(f"================================{i+1}==================================")
    model = HeteroGTFineTune(tokenizer=tokenizer, token_types=config.token_type, d_model=config.d_model, num_heads=config.num_heads, layer_types=['gnn', 'tf', 'gnn', 'tf'], max_num_adms=config.max_num_adms, 
                     device=device, task=curr_task, label_vocab_size=config.label_vocab_size, attn_mask_dicts=config.attn_mask_dicts,
                     use_cls_cat=True).to(device)
    if config.use_pretrained_model:
        model.load_weight(state_dict)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr)
    best_test_metric, best_test_long_seq_metric = train_with_early_stopping(model, train_dataloader, val_dataloader, test_dataloader,
                                                                            optimizer, loss_fn, device, config.early_stop_patience, task_type, config.epochs, 
                                                                            dec_loss_lambda=config.dec_loss_lambda, 
                                                                            val_long_seq_idx=val_long_seq_idx, test_long_seq_idx=test_long_seq_idx, 
                                                                            eval_metric=eval_metric, return_model=False)
    final_metrics.append(best_test_metric)
    final_long_seq_metrics.append(best_test_long_seq_metric)

================================1==================================


Running inference: 100%|██████████| 45/45 [00:02<00:00, 22.13it/s]


Validation: {'precision': 36.353, 'recall': 14.3667, 'f1': 19.0981, 'auc': 66.5156, 'prauc': 36.7519}
Test:      {'precision': 33.1821, 'recall': 13.7059, 'f1': 18.1904, 'auc': 67.2613, 'prauc': 35.7609}

Validation-long: {'precision': 36.6397, 'recall': 18.3494, 'f1': 22.0961, 'auc': 64.3429, 'prauc': 42.5343}
Test-long: {'precision': 31.8076, 'recall': 17.194, 'f1': 20.7733, 'auc': 65.1907, 'prauc': 40.4217}



Running inference: 100%|██████████| 45/45 [00:02<00:00, 22.01it/s]


Validation: {'precision': 41.6864, 'recall': 20.9485, 'f1': 25.4404, 'auc': 70.7111, 'prauc': 39.6872}
Test:      {'precision': 38.4604, 'recall': 19.8374, 'f1': 24.2126, 'auc': 72.4446, 'prauc': 39.2965}

Validation-long: {'precision': 46.6562, 'recall': 25.4735, 'f1': 29.2943, 'auc': 68.7802, 'prauc': 46.5703}
Test-long: {'precision': 41.2657, 'recall': 22.6928, 'f1': 26.8704, 'auc': 71.8297, 'prauc': 43.6032}



...

Validation: {'precision': 43.086, 'recall': 37.192, 'f1': 38.568, 'auc': 73.6229, 'prauc': 43.5202}
Test:      {'precision': 48.5667, 'recall': 36.2116, 'f1': 37.9616, 'auc': 73.7783, 'prauc': 43.0383}

Validation-long: {'precision': 42.0405, 'recall': 40.3862, 'f1': 40.6932, 'auc': 70.256, 'prauc': 49.7891}
Test-long: {'precision': 40.4433, 'recall': 40.8527, 'f1': 40.3401, 'auc': 73.4243, 'prauc': 45.4957}



Running inference: 100%|██████████| 45/45 [00:02<00:00, 21.61it/s]


Validation: {'precision': 43.6746, 'recall': 36.69, 'f1': 37.8722, 'auc': 73.737, 'prauc': 43.5932}
Test:      {'precision': 39.8397, 'recall': 35.3115, 'f1': 36.6628, 'auc': 74.0654, 'prauc': 42.8674}

Validation-long: {'precision': 43.9152, 'recall': 39.4518, 'f1': 40.8304, 'auc': 70.1474, 'prauc': 50.2605}
Test-long: {'precision': 41.0402, 'recall': 36.8123, 'f1': 38.273, 'auc': 73.5808, 'prauc': 45.4947}



Running inference: 100%|██████████| 45/45 [00:02<00:00, 21.93it/s]

Validation: {'precision': 42.6193, 'recall': 37.1741, 'f1': 38.3178, 'auc': 73.2436, 'prauc': 43.0524}
Test:      {'precision': 45.0786, 'recall': 36.3465, 'f1': 37.7212, 'auc': 73.7083, 'prauc': 42.3841}

Validation-long: {'precision': 41.993, 'recall': 40.4108, 'f1': 40.5588, 'auc': 70.7476, 'prauc': 49.898}
Test-long: {'precision': 39.8843, 'recall': 39.346, 'f1': 39.1917, 'auc': 72.9955, 'prauc': 45.9408}


Early stopping triggered (no improvement for 5 epochs).

Best validation performance:
{'global': {'precision': 47.3439, 'recall': 36.347, 'f1': 38.5767, 'auc': 74.4262, 'prauc': 44.7539}, 'per_class':                                            precision   recall       f1  \
Acute and unspecified renal failure          51.6854  52.9954  52.3322   
Acute cerebrovascular disease                 0.0000   0.0000   0.0000   
Acute myocardial infarction                   0.0000   0.0000   0.0000   
Cardiac dysrhythmias                         77.4127  68.7956  72.8502   
Chronic kidney

In [18]:
def topk_avg_performance_formatted(performances, long_seq_performances, k=5):
    metrics = ["f1", "auc", "prauc"]
    scores = {m: np.array([p[m] for p in performances]) for m in metrics}

    # 计算排名（值越大排名越靠前）
    ranks = {m: (-scores[m]).argsort().argsort() + 1 for m in metrics}
    avg_ranks = np.mean(np.stack([ranks[m] for m in metrics], axis=1), axis=1)

    # 选 top-k
    topk_idx = np.argsort(avg_ranks)[:k]
    final_avg = {m: np.mean([performances[i][m] for i in topk_idx]) for m in performances[0].keys()}
    final_std = {m: np.std([performances[i][m] for i in topk_idx], ddof=0) for m in performances[0].keys()}
    final_long_seq_avg = {m: np.mean([long_seq_performances[i][m] for i in topk_idx]) for m in long_seq_performances[0].keys()}
    final_long_seq_std = {m: np.std([long_seq_performances[i][m] for i in topk_idx], ddof=0) for m in long_seq_performances[0].keys()}

    # 打印结果（转百分比，均保留两位小数）
    print("Final Metrics:")
    for m in performances[0].keys():
        mean_val = final_avg[m]
        std_val = final_std[m]
        print(f"{m}: {mean_val:.2f} ± {std_val:.2f}")
    print("\nFinal Long Sequence Metrics:")
    for m in long_seq_performances[0].keys():
        mean_val = final_long_seq_avg[m]
        std_val = final_long_seq_std[m]
        print(f"{m}: {mean_val:.2f} ± {std_val:.2f}")

In [30]:
def print_per_class_performance(dfs, col_name="prauc"):
    """
    输入一个 DataFrame 列表，对每个疾病在所有表格的指定列计算 mean ± std 并打印。

    参数:
        dfs (list[pd.DataFrame]): 多个表格组成的列表
        col_name (str): 要计算的指标列名 (默认: "prauc")
    """
    # 拼接所有表格
    all_values = pd.concat(dfs, axis=0)

    # 按疾病分组，计算 mean 和 std
    grouped = all_values.groupby(all_values.index)[col_name].agg(["mean", "std"])

    # 打印
    for disease, row in grouped.iterrows():
        mean_val = row["mean"]
        std_val = row["std"]
        print(f"{disease}: {mean_val:.2f} ± {std_val:.2f}")

In [31]:
if task_type == "binary":
    topk_avg_performance_formatted(final_metrics, final_long_seq_metrics)
else:
    final_metrics_global = [metrics["global"] for metrics in final_metrics]
    final_metrics_per_class = [metrics["per_class"] for metrics in final_metrics]
    final_long_seq_metrics_global = [metrics["global"] for metrics in final_long_seq_metrics]
    final_long_seq_metrics_per_class = [metrics["per_class"] for metrics in final_long_seq_metrics]
    topk_avg_performance_formatted(final_metrics_global, final_long_seq_metrics_global)
    print("\nPer-class performance, all patients:")
    print_per_class_performance(final_metrics_per_class, col_name="prauc")
    print("\nPer-class performance, long seq:")
    print_per_class_performance(final_long_seq_metrics_per_class, col_name="prauc")

Final Metrics:
precision: 43.53 ± 2.15
recall: 36.31 ± 0.96
f1: 38.11 ± 0.77
auc: 73.47 ± 1.93
prauc: 42.67 ± 1.80

Final Long Sequence Metrics:
precision: 42.76 ± 0.63
recall: 39.19 ± 0.29
f1: 40.04 ± 0.22
auc: 72.06 ± 2.47
prauc: 45.96 ± 1.59

Per-class performance, all patients:
Acute and unspecified renal failure: 45.65 ± 1.58
Acute cerebrovascular disease: 7.40 ± 3.99
Acute myocardial infarction: 15.76 ± 5.01
Cardiac dysrhythmias: 74.58 ± 3.39
Chronic kidney disease: 79.53 ± 0.77
Chronic obstructive pulmonary disease: 44.01 ± 8.65
Conduction disorders: 4.93 ± 0.86
Congestive heart failure; nonhypertensive: 72.27 ± 3.32
Coronary atherosclerosis and related: 58.88 ± 2.43
Disorders of lipid metabolism: 54.73 ± 4.33
Essential hypertension: 62.66 ± 0.17
Fluid and electrolyte disorders: 49.45 ± 0.18
Gastrointestinal hemorrhage: 11.33 ± 2.68
Hypertension with complications: 71.66 ± 2.02
Other liver diseases: 3.08 ± 1.60
Other lower respiratory disease: 57.31 ± 4.18
Pneumonia: 17.63 ± 0.1